<a href="https://colab.research.google.com/github/formallinguist/NLP-projects./blob/main/Recommendation_System_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Recommendation System.
-Collaborative filtering.
    -it considers other data.
-Content based filtering.
    -Based on user data

Collaborative filtering.
-Memory based.
   -similarity,cosine similarity etc(Pandas,correlation)
-Model based.
  -KNN,SVD..etc

Popularity based.
-Fast and no personali

In [1]:
import pandas as pd
import os as io
import numpy as np
from pandas import Series,DataFrame,read_table
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error
%matplotlib inline

In [4]:
data = pd.read_csv('/content/ratings.csv')

In [5]:
data

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [6]:
#group the df by movie id to get the count of users watched.
train_g = data.groupby(['movieId'])['userId'].count().reset_index()

In [7]:
#rename the colomns
train_g.rename(columns={'userId':'score'},inplace=True)

In [8]:
 #sort the score to get the top rated movie.
 train_d = train_g.sort_values(['score','movieId'],ascending=False)

In [9]:
#Apply the ranking function to assign the rank based on the score.
train_d['rank'] = train_d['score'].rank(ascending=0,method='first')

In [10]:
train_d.head(4)

,movieId,score,rank
314,356,329,1.0
277,318,317,2.0
257,296,307,3.0
510,593,279,4.0


COLLABORATIVE FILTERING:

In [11]:
movie_rating =  data.pivot_table(index='userId',columns='movieId',values='rating')

In [12]:
starRating=movie_rating[356]

In [13]:
similar = (movie_rating.corrwith(starRating).dropna())

In [14]:
df = pd.DataFrame(movie_rating.corrwith(starRating).dropna())

In [15]:
movie_stats=data.groupby('movieId').agg({'rating':[np.size,np.mean]})

In [16]:
top = movie_stats.sort_values([('rating','mean')],ascending=False)

In [17]:
popular=movie_stats['rating','size']>=100

In [18]:
movie_stats[popular].sort_values([('rating','mean')],ascending=False) [:10]

rating          
          size      mean
movieId                 
318      317.0  4.429022
858      192.0  4.289062
2959     218.0  4.272936
1221     129.0  4.259690
48516    107.0  4.252336
1213     126.0  4.250000
912      100.0  4.240000
58559    149.0  4.238255
50       204.0  4.237745
1197     142.0  4.232394

In [19]:
matched = movie_stats[popular].join(DataFrame(similar,columns=['similarity']))

In [20]:
matched.sort_values('similarity',ascending=False)

,"(rating, size)","(rating, mean)",similarity
movieId,,,
356,329.0,4.164134,1.000000
1704,141.0,4.078014,0.484042
588,183.0,3.792350,0.464268
2329,129.0,4.217054,0.457287
1682,125.0,3.812000,0.432556
...,...,...,...
6874,131.0,3.961832,-0.009399
7153,185.0,4.118919,-0.016837
541,124.0,4.100806,-0.017542


MODEL BASED.

In [21]:
pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 5.4 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1623270 sha256=afd3e21b4f81ef539acc28ac91a0fdd4a4dbdd750cad88b191bfbacff53b1916
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [22]:
import surprise

In [23]:
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import KNNBasic,Reader,Dataset,SVD

In [24]:
Text = pd.read_csv('/content/ratings.csv')

In [25]:
reader=Reader()

In [26]:
Text.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [27]:
data = Dataset.load_from_df(Text[['userId','movieId','rating']],reader)

In [28]:
svd = SVD()

In [29]:
#Run five fold cross validation and print results.
cross_validate(svd,data,measures=['RMSE','MAE'],cv=5,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8649  0.8815  0.8724  0.8687  0.8746  0.8724  0.0056  
MAE (testset)     0.6658  0.6771  0.6689  0.6690  0.6719  0.6705  0.0038  
Fit time          4.87    4.78    6.52    11.05   4.74    6.39    2.42    
Test time         0.26    0.17    0.32    0.33    0.17    0.25    0.07    


{'fit_time': (4.870485782623291,
  4.776890277862549,
  6.515397310256958,
  11.048343896865845,
  4.74128532409668),
 'test_mae': array([0.66579315, 0.67705604, 0.66885443, 0.66903569, 0.67190147]),
 'test_rmse': array([0.8648994 , 0.88154996, 0.87241394, 0.86869455, 0.87463694]),
 'test_time': (0.2572753429412842,
  0.1699838638305664,
  0.32028865814208984,
  0.33376049995422363,
  0.1663832664489746)}

In [30]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [31]:
title = Text[['movieId']].drop_duplicates()

In [32]:
title.head(4)

,movieId
0,1
1,3
2,6
3,47


In [33]:
title['Estimate_Score']=title['movieId'].apply(lambda x: svd.predict(288,x).est)

In [34]:
svd.predict(288,1).est

4.001649443745945

In [35]:
title

,movieId,Estimate_Score
0,1,4.001649
1,3,3.240450
2,6,3.764308
3,47,3.759729
4,50,4.130948
...,...,...
100820,160341,3.045689
100821,160527,3.218864
100823,160836,3.340557
100827,163937,3.191574


In [36]:
title.sort_values('Estimate_Score',ascending=False)

,movieId,Estimate_Score
68,1196,4.546461
1453,858,4.545888
67,1136,4.511382
232,318,4.483547
28,527,4.458981
...,...,...
14369,1760,1.900393
1929,2412,1.766667
8483,3593,1.695352
26886,5323,1.640933
